# Replotting of several QC parameters

In [2]:
library(lattice)
library(tidyverse)
library(Seurat)
library(data.table)


载入程辑包：‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [3]:
whole <- readRDS("./data/whole_annot.rds")

## Figure 1F

In [4]:
library(speckle)

In [5]:
propel <- propeller(
    whole, clusters = whole$desc,
    group = whole$group
)
propel

extracting sample information from Seurat object

Performing logit transformation of proportions

group variable has > 2 levels, ANOVA will be performed



,BaselineProp,PropMean.H,PropMean.OC,PropMean.OD,Fstatistic,P.Value,FDR
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Mesothelial Cells,0.150753644,0.345579477,0.060289965,0.089400228,9.82969616,0.001501298,0.02402077
Fibro-inflammatory Progenitors,0.044374364,0.020802528,0.068579308,0.038579180,3.91558271,0.040268669,0.32214935
Proliferating Cells,0.004469718,0.007507453,0.002185971,0.003455529,3.22229450,0.065480573,0.34922972
Adipocytes,0.007101885,0.002749715,0.006452649,0.010530354,2.48190185,0.113769792,0.40540998
Smooth Muscle Cells,0.003550942,0.001719584,0.004032611,0.004470252,2.33199198,0.127818059,0.40540998
Adipose Progenitor Cells,0.249012937,0.156614820,0.275891555,0.292764110,2.11248437,0.152028742,0.40540998
Th1,0.118968985,0.072792782,0.117279274,0.150953915,1.69176230,0.214205045,0.47550947
Mast Cells,0.011223958,0.029018310,0.009938894,0.005284203,1.61816770,0.237754734,0.47550947
Cytotoxic T cells,0.109955055,0.071141383,0.101599806,0.144320232,1.34355459,0.287644651,0.51136827


In [8]:
library(ComplexHeatmap)
library(RColorBrewer)
library(circlize)

circlize version 0.4.15
CRAN page: https://cran.r-project.org/package=circlize
Github page: https://github.com/jokergoo/circlize
Documentation: https://jokergoo.github.io/circlize_book/book/

If you use it in published research, please cite:
Gu, Z. circlize implements and enhances circular visualization
  in R. Bioinformatics 2014.

This message can be suppressed by:
  suppressPackageStartupMessages(library(circlize))




In [12]:
hm_mat <- propel[,c(2:4)]
colnames(hm_mat) <- c("LC", "OC", "OD")

In [14]:
blues <- colorRamp2(seq(min(hm_mat), max(hm_mat), length.out = 9), brewer.pal(9, "Blues"))

In [25]:
row_bar <- data.frame(
    row.names = rownames(propel),
    logP = -log10(propel$FDR)
) %>% deframe()

In [28]:
svg("./plots/replot/prop_hm.svg", width = 6, height = 8)
Heatmap(
    hm_mat,
    col = blues,
    cluster_columns = FALSE,
    cluster_rows = FALSE,
    row_names_side = "left",
    name = "Proportion",
    right_annotation = rowAnnotation(
        logP = anno_barplot(row_bar)
    )
)
dev.off()

Warning message:
“The input is a data frame-like object, convert it to a matrix.”


png 
  2

## Average cell count

In [29]:
count_tbl <- table(whole$desc, whole$group) %>%
    as.data.table()
count_tbl %>% head()

V1,V2,N
<chr>,<chr>,<int>
Adipose Progenitor Cells,H,1611
Mesothelial Cells,H,3939
Th1,H,873
Cytotoxic T cells,H,855
gamma-delta T,H,748
M2-like Macrophages,H,943


In [47]:
svg("./plots/replot/mean_count.svg", width = 8, height = 6)
with(
    count_tbl[,.(cell_type = as.factor(V1),
                 group = as.factor(V2),
            mean_count = ifelse(
                V2 == "H", N / 2, N / 3
            ))][,.(
        cell_type = reorder(cell_type, mean_count),
        group, mean_count
    )],
    barchart(
        cell_type ~ mean_count, groups = group,
        par.settings = list(
            superpose.polygon = list(
                col = rev(brewer.pal(3, "Set1"))
            )
        ),
        auto.key = list(columns = 3),
        xlab = "Mean count",
        xlim = c(0, 2000)
    )
)
dev.off()

png 
  2

## Cell Capture

In [65]:
sample <- unique(whole$sample)
sample <- str_replace(
    sample, "OC", "ON"
)
sample <- str_replace(
    sample, "OD", "OT"
)

In [67]:
cap_tbl <- data.table(
    sample = sample,
    raw = map_dbl(
        sample,
        \(x){
            nrow(fread(
            sprintf(
                "~/bioresource/SVF_dataset/data/%s/doublets_barcodes.csv",
                x)))
        }
    ),
    filtered = dplyr::count(whole@meta.data, sample)$n
)
cap_tbl

sample,raw,filtered
<chr>,<dbl>,<int>
H12,13875,7333
H13,53984,3883
ON305,13893,3679
ON315,9598,2765
ON330,11058,5444
OT287,9052,4562
OT294,14836,6602
OT306,10845,6003


In [83]:
svg("./plots/replot/cell_cap.svg", width = 8, height = 6)
with(
    melt(cap_tbl, id.vars = c("sample"),
         variable.name = "state", value.name = "count"),
    barchart(
        sample ~ count, groups = state,
        par.settings = list(
            superpose.polygon = list(
                col = rev(brewer.pal(3, "Set2"))[c(2, 3)]
            )
        ),
        xlab = "Cell Count",
        auto.key = list(columns = 2)
    )
)
dev.off()

Warning message in melt.data.table(cap_tbl, id.vars = c("sample"), variable.name = "state", :
“'measure.vars'[raw, filtered] 的数据类型不完全相同。由于融合后的数据将是'double'类型，所以变量中不是'double'类型的数将被强制转换为'double'类型，更多关于强制转换的信息请查看 ?melt.data.table.”


png 
  2

## Cell Cycle

In [84]:
whole <- CellCycleScoring(
    whole,
    s.features = cc.genes$s.genes,
    g2m.features = cc.genes$g2m.genes
)

Warning message:
“The following features are not present in the object: MLF1IP, not searching for symbol synonyms”
Warning message:
“The following features are not present in the object: FAM64A, HN1, not searching for symbol synonyms”


In [85]:
count_tbl <- table(whole$desc, whole$Phase) %>%
    as.data.table()
count_tbl %>% head()

V1,V2,N
<chr>,<chr>,<int>
Adipose Progenitor Cells,G1,5017
Mesothelial Cells,G1,3151
Th1,G1,1196
Cytotoxic T cells,G1,1014
gamma-delta T,G1,641
M2-like Macrophages,G1,2180


In [89]:
svg("./plots/replot/cell_cycle.svg", width = 8, height = 6)
with(
    count_tbl[,.(phase = as_factor(V2), prop = N / sum(N)),
              by = V1][,.(
        cell_type = reorder(V1, prop),
        phase, prop
    )],
    barchart(
        cell_type ~ prop, groups = phase,
        par.settings = list(
            superpose.polygon = list(
                col = rev(brewer.pal(3, "Set1"))
            )
        ), stack = TRUE,
        auto.key = list(columns = 3),
        xlab = "Proportion",
        xlim = c(0, 1)
    )
)
dev.off()

png 
  2

## QC params

In [101]:
whole_qc <- melt(whole@meta.data[,c("nFeature_RNA",
                       "nCount_RNA", "mt_percentage")],
    variable.name = "param")

Warning message in melt(whole@meta.data[, c("nFeature_RNA", "nCount_RNA", "mt_percentage")], :
“data.table 中的 melt 泛型函数被传递了 data.frame 即将转向到 reshape2 中的相关方法；请注意 reshape2 已经弃用，这个转向也已经弃用。如果要在 data.table 和 reshape2 同时附着的情况下继续使用 reshape2 中的 melt 方法，（例如 melt.list），你可以把命名空间写在函数名称前面，例如 reshape2::melt(whole@meta.data[, c("nFeature_RNA", "nCount_RNA", "mt_percentage")])。在下一个版本中，此警告将变成为错误。”
No id variables; using all as measure variables



In [113]:
png("./plots/replot/qc_params.png", width = 18,
    height = 6, res = 300, units = "in")
with(
    whole_qc,
    densityplot(
        ~ value| param,
        scale = list(
            x = "free",
            y = "free"
        ),
        layout = c(3, 1)
    )
)
dev.off()

png 
  2